In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%cd ..

/scratch/km817/New_iREC/iREC


In [4]:
import torch
import torch.distributions as dist
import math
import matplotlib.pyplot as plt
import pickle as pkl

from tqdm.notebook import tqdm
from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.EmpiricalMixturePosterior import EmpiricalMixturePosterior
from rec.beamsearch.samplers.GreedySampling import GreedySampler
from rec.beamsearch.Coders.Encoder import Encoder as Empirical_Encoder
from models.SimpleBayesianLinRegressor import BayesLinRegressor
from rec.utils import kl_estimate_with_mc, plot_samples_in_2d, plot_running_sum_2d, plot_pairs_of_samples
from rec.OptimisingVars.FinalJointOptimiser import FinalJointOptimiser

In [5]:
torch.set_default_tensor_type(torch.DoubleTensor)

In [6]:
def encode_sample(target, omega=5, epsilon=0.,
                  n_empirical_samples=10, seed=0, beamwidth=1, optimising_vars=False, aux_vars=None, dont_run=False):
    
    encoder = Empirical_Encoder(target,
                                seed,
                                CodingSampler,
                                GreedySampler,
                                EmpiricalMixturePosterior,
                                omega,
                                n_empirical_samples,
                                epsilon=epsilon,
                                beamwidth=beamwidth
                                )
    if aux_vars is not None:
        encoder.auxiliary_posterior.coding_sampler.auxiliary_vars = aux_vars
    
    if dont_run:
        return encoder
    else:
        return encoder, *encoder.run_encoder()

In [7]:
def create_blr_problem(dim, seed):
    if dim in [2, 5]:
        signal_std = 1e-3
    else:
        signal_std = 1e-1
    torch.set_default_tensor_type(torch.DoubleTensor)
    blr = BayesLinRegressor(prior_mean=torch.zeros(dim),
                            prior_alpha=1,
                            signal_std=signal_std,
                            num_targets=100,
                            seed=1)
    blr.sample_feature_inputs()
    blr.sample_regression_targets()
    blr.posterior_update()
    target = blr.weight_posterior
    return blr, target

In [80]:
dim = 90
omega = 5
blr_seed = 1
b, t = create_blr_problem(dim=dim, seed=blr_seed)
num_compressed_samples = 50

In [81]:
z_sample = t.mean
compute_params_enc = encode_sample(target=t, dont_run=True)
n_auxiliaries = compute_params_enc.n_auxiliary
kl_q_p = compute_params_enc.total_kl
optimising = FinalJointOptimiser(z_sample, omega, n_auxiliaries, kl_q_p, n_trajectories=100, total_var=1)
aux_vars = optimising.run_optimiser(epochs=1000)
pkl.dump(aux_vars, open(f"PickledStuff/Correlated/Dim{dim}/optimised_vars_emp.pkl", "wb"))

The mean loss is 4.22685. The mean KL is: 5.13326: 100%|██████████| 1000/1000 [12:56<00:00,  1.29it/s]  


FileNotFoundError: [Errno 2] No such file or directory: 'PickledStuff/Correlated/Dim90/optimised_vars_emp.pkl'

In [82]:
pkl.dump(aux_vars, open(f"PickledStuff/Correlated/Dim{dim}/optimised_vars_emp.pkl", "wb"))

In [13]:
dim = 25
omega = 5
blr_seed = 1
b, t = create_blr_problem(dim=dim, seed=blr_seed)
num_compressed_samples = 50

In [14]:
beamwidth = 20
torch.manual_seed(0)
seeds = torch.randint(low = 0, high = int(1e6), size=(num_compressed_samples,))
epsilons = [0., 0.05, 0.1, 0.15, 0.2]
aux_vars = torch.tensor(pkl.load(open(f"PickledStuff/Correlated/Dim{dim}/approx_vars_emp.pkl", "rb")))
for eps in epsilons:
    exp_dict = {}
    exp_dict['seeds'] = seeds.numpy()
    exp_dict['target_mean'] = t.mean.numpy()
    exp_dict['target_covar'] = t.covariance_matrix.numpy()
    exp_dict['compressed_samples'] = []
    exp_dict['compressed_samples_idxs'] = []
    exp_dict['aux_vars'] = aux_vars
    pbar = tqdm(enumerate(seeds), total=num_compressed_samples)
    log_probs = torch.zeros([0])
    for i, s in pbar:
        enc, z, idx = encode_sample(target=t, beamwidth=beamwidth, epsilon=eps, omega=omega, 
                                    seed=s, n_empirical_samples=50, aux_vars=aux_vars)
        idxs_to_transmit = idx[0]
        best_sample = z[0]
        log_probs = torch.cat((log_probs, t.log_prob(best_sample)[None]))
        exp_dict['compressed_samples'].append(best_sample.numpy())
        exp_dict['compressed_samples_idxs'].append(idxs_to_transmit.numpy())
        pbar.set_description(f"Coded sample {i + 1}, has log prob of {t.log_prob(best_sample)}")
    
    print(torch.mean(log_probs))
    with open(f"PickledStuff/CorrelatedApprox/Dim{dim}/Empirical_Epsilon{eps}_Beam{beamwidth}_Omega{omega}.pkl", "wb") as f:
        pkl.dump(exp_dict, f)

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(-276.7617)


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(-138.6015)


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(-54.0020)


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(-13.5350)


  0%|          | 0/50 [00:00<?, ?it/s]

tensor(21.7857)


In [12]:
print('done')

done
